In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"boochanggyu","key":"b4302904daddc3ed444c4f3c20a65c32"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c hms-harmful-brain-activity-classification

100% 18.4G/18.4G [07:54<00:00, 43.4MB/s]
100% 18.4G/18.4G [07:54<00:00, 41.7MB/s]


In [ ]:
!ls

drive  hms-harmful-brain-activity-classification.zip  kaggle.json  sample_data


In [ ]:
!unzip /content/hms-harmful-brain-activity-classification.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train_spectrograms/2080462125.parquet  
  inflating: train_spectrograms/208059969.parquet  
  inflating: train_spectrograms/2080686415.parquet  
  inflating: train_spectrograms/2080973847.parquet  
  inflating: train_spectrograms/2081117549.parquet  
  inflating: train_spectrograms/2081307623.parquet  
  inflating: train_spectrograms/2081826095.parquet  
  inflating: train_spectrograms/2081944512.parquet  
  inflating: train_spectrograms/2082016410.parquet  
  inflating: train_spectrograms/2082370788.parquet  
  inflating: train_spectrograms/2082376822.parquet  
  inflating: train_spectrograms/2082907673.parquet  
  inflating: train_spectrograms/2083007837.parquet  
  inflating: train_spectrograms/2083142969.parquet  
  inflating: train_spectrograms/2083155730.parquet  
  inflating: train_spectrograms/208321640.parquet  
  inflating: train_spectrograms/2083642063.parquet  
  inflating: train_spectrograms/2084695153.parquet  
  inflating:

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import gc
from sklearn.model_selection import KFold
import torch.nn.init as init
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, SubsetRandomSampler
from urllib.request import urlopen

In [ ]:
import random

seed = 40
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
warnings.filterwarnings('ignore')

data load

In [ ]:
df_train = pd.read_csv('/content/train.csv')

In [ ]:
offset = df_train['spectrogram_label_offset_seconds']

In [ ]:
target = df_train.columns[-6:]
non_target = df_train.columns[:-6]
x = df_train[non_target]
y = df_train[target].values
sum_y = y.sum(axis = 1 ,keepdims=True)
y = y / sum_y
target = y

# x는 pd 형식의 훈련데이터, y는 value만 뽑아냄

In [ ]:
eeg_dir = "/content/train_eegs"
spectrogram_dir = "/content/train_spectrograms"

preprocessing

In [ ]:
train_x = x.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first'})
train_x.columns = ['spec_id']

tmp = x.groupby('eeg_id')[['patient_id']].agg('first') # The code adds the patient_id for each eeg_id to the train DataFrame. This links each EEG segment to a specific patient.
train_x['patient_id'] = tmp

target_column = ['seizure_vote' ,'lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
tmp = df_train.groupby('eeg_id')[target_column].agg('sum')
for t in target_column:
    train_x[t] = tmp[t].values

y = train_x[target_column].values
y = y / y.sum(axis = 1, keepdims = True)
train_x[target_column] = y

tmp = x.groupby('eeg_id')[['expert_consensus']].agg('first')
train_x['target'] = tmp

train_x = train_x.reset_index()

In [ ]:
train_x['eeg_path'] = train_x['eeg_id'].apply(lambda x: os.path.join(eeg_dir, f'{x}.parquet'))
train_x['spectrogram_path'] = train_x['spec_id'].apply(lambda x: os.path.join(spectrogram_dir, f'{x}.parquet'))

In [ ]:
train_x['patient_id'] = (train_x['patient_id'] - train_x['patient_id'].mean()) / train_x['patient_id'].std()
x = train_x
x = x.fillna(0)

In [ ]:
x['Seizure'] = 0
x['GPD'] = 0
x['LPD'] = 0
x['LPDA'] = 0
x['GPDA'] = 0
x['Other'] = 0

x.loc[x['target'] == 'Seizure', 'Seizure'] = 1
x.loc[x['target'] == 'GPD', 'GPD'] = 2
x.loc[x['target'] == 'LPD' , 'LPD'] = 3
x.loc[x['target'] == 'LPDA' , 'LPDA'] = 4
x.loc[x['target'] == 'GPDA' , 'GPDA'] = 5
x.loc[x['target'] == 'Other' , 'Other'] = 6

x['answer'] = x['target']

x['target'] = x['Seizure'] + x['GPD'] + x['LPD'] + x['LPDA'] + x['GPDA'] + x['Other']

x.drop(['Seizure' , 'GPD' , 'LPD' , 'LPDA' , 'GPDA' , 'Other'] , axis = 1 , inplace = True)

x['eeg_id'] = (x['eeg_id'] - x['eeg_id'].mean()) / (x['eeg_id'].std() + 0.0000000001)
x['spec_id'] = (x['spec_id'] - x['spec_id'].mean()) / (x['spec_id'].std() + 0.0000000001)

data loader func

In [ ]:
def load_spectrogram_data(spec_path):
    if os.path.isfile(spec_path):
        spec_data = pd.read_parquet(spec_path)
        return spec_data
    else:
        print(f"Invalid file path: {spec_path}")
        return None

In [ ]:
class data_loader(Dataset):
    def __init__(self,imgs, targets):
        self.imgs=imgs
        self.targets=targets
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self , idx):
        image_path=self.imgs[idx]
        target_path=self.targets[idx]

        return image_path , target_path

In [ ]:
class Spec():
    def __init__(self, start , batch):
        self.start = start
        self.batch = batch

    def spec_input(self):
        result = []
        target = []
        columns = ['seizure_vote' , 'lpd_vote' , 'gpd_vote' , 'lrda_vote' , 'grda_vote', 'other_vote']

        rc = load_spectrogram_data(x['spectrogram_path'][0]).columns
        rc = rc[1:401]


        for i in range(self.start , self.start + self.batch):
            idx = i - self.start
            result.append(0)
            result[idx] = load_spectrogram_data(x['spectrogram_path'][i])
            target.append(0)
            target[idx] = x.loc[i][columns]
            #target.append(x.loc[i , ['answer']])

            target[idx] = target[idx].values
            target[idx] = np.array(target[idx])
            target[idx] = target[idx].tolist()
            target[idx] = np.array(target[idx])

            # spec data 만들기
            sp_offset = int(offset[i])
            sp = result[idx].loc[(result[idx].time>=sp_offset)&(result[idx].time<sp_offset+600)]
            result[idx] = sp

            result[idx] = result[idx].iloc[: , 1:401]

            result[idx] = result[idx].fillna(0)
            #result[idx] = result[idx].iloc[:300 , 1:101]
            result[idx] = result[idx].values
            result[idx] = np.array(result[idx])

        return result , target

In [ ]:
spec = Spec(0 , 15100)
image , label_y = spec.spec_input()

In [ ]:
def making_dataset(image , label_y):
  target= 0
  target = torch.tensor(label_y)
  target = np.array(target)
  for i in range(len(target)):
    target[i] = np.array(target[i])
  target = target.tolist()

  result = []
  for i in range(len(image)):
    result.append(0)
    im = torch.tensor(image[i])
    result[i] = torch.log(im + 0.0000000000000001)
    result[i] = np.array(result[i])

  for j in range(len(image)):
    i = len(image) - j
    if len(result) >= i+1:
      if len(target) >= i+1:
        if result[i].shape[0] < 1:
          del result[i]
          del target[i]

  transform = transforms.Compose([
    transforms.Resize((672,224)),
    #transforms.RandomHorizontalFlip(p=0.3),
    #transforms.RandomVerticalFlip(p=0.3),
  ])

  for i in range(len(result)):
    result[i] = torch.tensor(result[i])
    result[i] = result[i].unsqueeze(0)
    result[i] = torch.tensor(result[i])
    target[i] = torch.tensor(target[i])

  for i in range(len(result)):
    result[i] = transform(result[i])
    result[i] = result[i].reshape(3,224,224)

  return result , target # result.shape : (3,224,224)

In [ ]:
data,target=making_dataset(image,label_y)

데이터 augmentation

In [ ]:
result0 = result[:11100]
target0 = target[:11100]
result1 = result[11101:]
target1 = target[11101:]

In [ ]:
print(len(result) , len(target))

13275 13275


In [ ]:
train_dataset = data_loader(result0, target0) # result.shape : batch_size , 3, 224 , 224
train_dataloader= DataLoader(train_dataset,batch_size=64,shuffle=True)
valid_dataset = data_loader(result1,target1)
valid_dataloader = DataLoader(valid_dataset,batch_size=64,shuffle=True)
dataset = data_loader(result,target)
dataloader = DataLoader(dataset,batch_size=64,shuffle=True)

*conv_block* 만들기

In [ ]:
!pip install timm
import timm

In [ ]:
vit_models = [model_name for model_name in timm.list_models(pretrained=True) if 'efficientvit' in model_name]
print(vit_models)

['efficientvit_b0.r224_in1k', 'efficientvit_b1.r224_in1k', 'efficientvit_b1.r256_in1k', 'efficientvit_b1.r288_in1k', 'efficientvit_b2.r224_in1k', 'efficientvit_b2.r256_in1k', 'efficientvit_b2.r288_in1k', 'efficientvit_b3.r224_in1k', 'efficientvit_b3.r256_in1k', 'efficientvit_b3.r288_in1k', 'efficientvit_l1.r224_in1k', 'efficientvit_l2.r224_in1k', 'efficientvit_l2.r256_in1k', 'efficientvit_l2.r288_in1k', 'efficientvit_l2.r384_in1k', 'efficientvit_l3.r224_in1k', 'efficientvit_l3.r256_in1k', 'efficientvit_l3.r320_in1k', 'efficientvit_l3.r384_in1k', 'efficientvit_m0.r224_in1k', 'efficientvit_m1.r224_in1k', 'efficientvit_m2.r224_in1k', 'efficientvit_m3.r224_in1k', 'efficientvit_m4.r224_in1k', 'efficientvit_m5.r224_in1k']


In [ ]:
model_name = "efficientvit_b0.r224_in1k"
model = timm.create_model(model_name, pretrained=True)

In [ ]:
input = 1024
for param in model.parameters():
  param_requires_grad = True
del model.head.classifier
model.head.classifier = nn.Linear(input,input)
for param in model.head.classifier.parameters():
  param_requires_grad = True

In [ ]:
class Egg(nn.Module):
  def __init__(self , input_size = 1024):
    super().__init__()
    self.input_size = input_size
    self.backbone = model
    self.softmax = nn.Softmax(dim=1)
    self.leakyrelu = nn.LeakyReLU()
    self.dropout = nn.Dropout(0.2)

    self.l1 = nn.Linear(224 , 1)
    self.l2=nn.Linear(224,6)

    self.convt1 = nn.ConvTranspose2d(1,1,kernel_size=3,stride=2,padding=3)
    self.convt2 = nn.ConvTranspose2d(1,1,kernel_size=3,stride=2,padding=3)
    self.convt3 = nn.ConvTranspose2d(1,1,kernel_size=6,stride=2,padding=3)

    self.b = nn.BatchNorm2d(1)

  def forward(self , x):
    vit = self.backbone(x)  # vit.shape : batch_num , 1024
    vit = vit.unsqueeze(1)
    vit = vit.unsqueeze(1)

    vit = vit.reshape(-1,1,32,32)

    vit=self.convt1(vit)
    vit=self.b(vit)
    vit=self.leakyrelu(vit)
    vit=self.convt2(vit)
    vit=self.b(vit)
    vit=self.leakyrelu(vit)
    vit=self.convt3(vit)
    vit=self.b(vit)
    vit=self.leakyrelu(vit)

    ssl = self.l1(vit)
    ssl = ssl.permute(0,1,3,2)
    ssl=self.l2(ssl)
    ssl=ssl.reshape(-1,6)
    ssl=self.softmax(ssl)

    return ssl , vit

[(h + 2p -d(k-1)-1)s + 1]

In [ ]:
model = model.to(device)
egg = Egg()
egg = egg.to(device)

In [ ]:
egg = Egg()
egg = egg.to('cpu')

In [ ]:
kl_loss = nn.KLDivLoss(reduction='batchmean')
mae_loss = nn.L1Loss()
mse_loss = nn.MSELoss().to(device)
softmax_loss = nn.Softmax()
criterion = nn.CrossEntropyLoss()

In [ ]:
def custome_loss(p,y,x):
  p1 , p2 = p

  l1 = kl_loss(p1.log() , y)

  l2 = torch.tensor(0.).to(device)
  cnt= 0
  for param in model.parameters():
    cnt+=1
    l2 += torch.norm(param)
  l2 = 0.002*l2
  l2 = l2/cnt

  l3 = mae_loss(p2 , x)

  return l1 + l2  + l3

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
optimizer = torch.optim.Adam(egg.parameters() , lr = 0.00005)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0.000000025)

In [ ]:
cnt = 0
loss_history = []
valid_loss_history = []
for epoch in range(400):
    egg.train()
    for images, labels in train_dataloader:
        images = torch.tensor(images)
        images = images.to(torch.float32)
        labels = torch.tensor(labels)
        labels = labels.to(torch.float32)

        if cnt == 0:
          images = images.to(device)
          labels = labels.to(device)

          #loss = kl_loss(egg(images).log() , labels[:,:6])
          loss = custome_loss(egg(images) , labels[:,:6] , images)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          gc.collect()
    scheduler.step()
    egg.eval()
    with torch.no_grad():
      for inputs, valid_labels in valid_dataloader:
        inputs = torch.tensor(inputs)
        inputs = inputs.to(torch.float32)
        valid_labels = torch.tensor(valid_labels)
        valid_labels = valid_labels.to(torch.float32)
      if cnt == 0:
        inputs = inputs.to(device)
        valid_labels = valid_labels.to(device)

        o1,o2 = egg(inputs)
        valid_loss = kl_loss(o1.log(), valid_labels[:,:6])
    if cnt == 0:
      print("epoch : {} , loss : {} , valid_loss : {} , optimizer : {}".format(epoch, loss.mean().item() , valid_loss.mean().item() ,  optimizer.param_groups[0]['lr']))

epoch : 0 , loss : 3.5331525802612305 , valid_loss : 1.3771964311599731 , optimizer : 4.8777024700925154e-05
epoch : 1 , loss : 5.543661117553711 , valid_loss : 1.2767754793167114 , optimizer : 4.5227812146944e-05


KeyboardInterrupt: 

cv

드라이브마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH = '/content/drive/MyDrive/'

torch.save(egg.state_dict(), PATH + 'eeg_state_dict.pt')
torch.save(model, PATH + 'model.pt')
#torch.save(egg, PATH + 'brain_model.pt')

추론

In [ ]:
test_spec_dir = '/content/test_spectrograms'
df_test = pd.read_csv('/content/test.csv')
df_test['spec_path'] = df_test['spectrogram_id'].apply(lambda x: os.path.join(test_spec_dir, f'{x}.parquet'))

In [ ]:
def test_load_spectrogram_data(spec_path):
    if os.path.isfile(spec_path):
        spec_data = pd.read_parquet(spec_path)
        return spec_data
    else:
        print(f"Invalid file path: {spec_path}")
        return None

In [ ]:
class test_Spec():
    def __init__(self, start , batch):
        self.start = start
        self.batch = batch

    def spec_input(self):
        result = []
        columns = ['seizure_vote' , 'lpd_vote' , 'gpd_vote' , 'lrda_vote' , 'grda_vote', 'other_vote']

        rc = test_load_spectrogram_data(df_test['spec_path'][0]).columns
        rc = rc[1:401]


        for i in range(self.start , self.start + self.batch):
            idx = i - self.start
            result.append(0)
            result[idx] = test_load_spectrogram_data(df_test['spec_path'][i])
            result[idx] = result[idx].iloc[: , 1:401]

            result[idx] = result[idx].fillna(0)
            #result[idx] = result[idx].iloc[:300 , 1:101]
            result[idx] = result[idx].values
            result[idx] = np.array(result[idx])

        return result

In [ ]:
def making_dataset(image):

  result = []
  for i in range(len(image)):
    result.append(0)
    im = torch.tensor(image[i])
    result[i] = torch.log(im + 0.0000000000000001)
    result[i] = np.array(result[i])

  for j in range(len(image)):
    i = len(image) - j
    if len(result) >= i+1:
        if result[i].shape[0] < 1:
          del result[i]

  transform = transforms.Compose([
    transforms.Resize((672,224)),
  ])

  for i in range(len(result)):
    result[i] = torch.tensor(result[i])
    result[i] = result[i].unsqueeze(0)
    result[i] = torch.tensor(result[i])

  for i in range(len(result)):
    result[i] = transform(result[i])
    if result[i].max() != result[i].min():
      result[i] = (result[i]-result[i].max())/(result[i].max() - result[i].min())
    else:
      result[i] = torch.zeros(672,224)
    result[i] = result[i].reshape(3,224,224)

  return result

In [ ]:
pred = egg.eval()
pred = pred.to(device)

dfl = df_test['spec_path'].shape[0]
np_test_preds = np.zeros((dfl , 6))

In [ ]:
for i in range(dfl):
    test_spec_fun = test_Spec(i , 1)
    test_spec = test_spec_fun.spec_input()
    test_spec = torch.tensor(test_spec)
    if test_spec.shape[0] > 0:
        test_spec = torch.tensor(test_spec)
        test_spec = test_spec.to(torch.float32)
        test_spec = making_dataset(test_spec)
        test_spec[0] = np.array(test_spec[0])
        test_spec = torch.tensor(test_spec)
        test_spec=test_spec.to(device)
        p1,p2 = pred(test_spec)
        pre = p1.tolist()
        for j in range(0,6):
            np_test_preds[i][j] = pre[0][j]

In [ ]:
np_test_preds

array([[0.3159436 , 0.13712355, 0.27599487, 0.01042787, 0.00676364,
        0.25374648]])